In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# Make a request to the website
r = requests.get('https://bola.kompas.com/')

In [3]:
# Create an object to parse the HTML format
soup = BeautifulSoup(r.content, 'html.parser')

In [4]:
# Retrieve all popular news links (Fig. 1)
link = []
for i in soup.find('div', {'class':'most__wrap'}).find_all('a'):
    i['href'] = i['href'] + '?page=all'
    link.append(i['href'])

In [5]:
# For each link, we retrieve paragraphs from it, combine each paragraph as one string, and save it to documents (Fig. 2)
documents = []
for i in link:
    # Make a request to the link
    r = requests.get(i)
  
    # Initialize BeautifulSoup object to parse the content 
    soup = BeautifulSoup(r.content, 'html.parser')
  
    # Retrieve all paragraphs and combine it as one
    sen = []
    for i in soup.find('div', {'class':'read__content'}).find_all('p'):
        sen.append(i.text)
  
    # Add the combined paragraphs to documents
    documents.append(' '.join(sen))

In [6]:
import re
import string
documents_clean = []
for d in documents:
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    document_test = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(document_test)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

docs = documents
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(docs)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())

In [8]:
import numpy as np
def get_similar_articles(q, df):
  print("query:", q)
  print("Berikut artikel dengan nilai cosine similarity tertinggi: ")
  # Convert the query become a vector
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  # Calculate the similarity
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  # Sort the values 
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # Print the articles and their similarity values
  for k, v in sim_sorted:
    if v != 0.0:
      print("Nilai Similaritas:", v)
      print(docs[k])
      print()
# Add The Query
q1 = 'barcelona'
# Call the function
get_similar_articles(q1, df)

query: barcelona
Berikut artikel dengan nilai cosine similarity tertinggi: 
Nilai Similaritas: 0.2572893059052084
KOMPAS.com - Kabar lanjutan dari grande partita  AC Milan vs  Juventus menjadi atensi dari pembaca  Bola.Kompas.com pada Kamis (7/1/2021). Bukan megabintang Cristiano Ronaldo, melainkan Sang Permata, Paulo Dybala, yang mencuri perhatian dalam kemenangan Juventus dengan skor 3-1 itu. Demikian diutarakan oleh legenda Juventus,  Alessandro Del Piero. Baca juga: Tampil Buruk Lawan AC Milan, Cristiano Ronaldo Panen Nilai 5 Lanjutan dari laga besar  AC Milan vs Juventus itu pun berlanjut terkait kinerja sang pengadil alias wasit di atas lapangan. Beberapa media Italia bahkan memberi penilaian jelek terhadap kinerja wasit asal Florence dalam partai tersebut. Adapun di luar laga dua raksasa Italia itu, di ranah Liga Spanyol, aksi Messi bersama  Barcelona kembali menjadi perhatian. Dalam kemenangan Barcelona atas Athletic Bilbao, Messi kembali membuat rekor. Berikut ini beberapa art

In [28]:
import itertools
import sys
from nltk.grammar import Nonterminal


def generate(grammar, start=None, depth=None, n=None):
    """
    Generates an iterator of all sentences from a CFG.

    :param grammar: The Grammar used to generate sentences.
    :param start: The Nonterminal from which to start generate sentences.
    :param depth: The maximal depth of the generated tree.
    :param n: The maximum number of sentences to return.
    :return: An iterator of lists of terminal tokens.
    """
    if not start:
        start = grammar.start()
    if depth is None:
        depth = sys.maxsize

    iter = _generate_all(grammar, [start], depth)

    if n:
        iter = itertools.islice(iter, n)

    return iter



def _generate_all(grammar, items, depth):
    if items:
        try:
            for frag1 in _generate_one(grammar, items[0], depth):
                for frag2 in _generate_all(grammar, items[1:], depth):
                    yield frag1 + frag2
        except RuntimeError as _error:
            if _error.message == "maximum recursion depth exceeded":
                # Helpful error message while still showing the recursion stack.
                raise RuntimeError(
                    "The grammar has rule(s) that yield infinite recursion!!"
                )
            else:
                raise
    else:
        yield []


def _generate_one(grammar, item, depth):
    if depth > 0:
        if isinstance(item, Nonterminal):
            for prod in grammar.productions(lhs=item):
                for frag in _generate_all(grammar, prod.rhs(), depth - 1):
                    yield frag
        else:
            yield [item]


demo_grammar = """
  S -> NP VP
  NP -> Det N
  PP -> P NP
  VP -> 'mencuri' | 'memberi' NP | mengukir'' PP
  Det -> 'sebuah' 
  N -> 'AC Milan' | 'Barcelona' | 'Juventus'
  P -> 'yang' | 'dengan'
"""


def demo(N=23):
    from nltk.grammar import CFG

    print("Generating the first %d sentences for demo grammar:" % (N,))
    print(docs)
    grammar = CFG.fromstring(docs)
    for n, sent in enumerate(generate(grammar, n=N), 1):
        print("%3d. %s" % (n, " ".join(sent)))



if __name__ == "__main__":
    demo()

Generating the first 23 sentences for demo grammar:
['KOMPAS.com -\xa0Kepemimpinan wasit Massimilano Irrati pada laga  AC Milan vs  Juventus semalam tengah menjadi sorotan. Beberapa media Italia bahkan memberi penilaian jelek terhadap kinerja wasit asal Florence dalam partai tersebut. La Gazzetta dello Sport dan Corriere dello Sport memberi nilai 4,5 untuk Irrati, sedangkan Tuttosport hanya 4. Irrati dinilai banyak membuat keputusan yang merugikan dalam grande partita pekan ke-16  Liga Italia 2020-2021 itu. Baca juga: Pahlawan Kemenangan Juventus atas AC Milan Mengaku Dihina oleh Wasit Kesalahan pertama Irrati adalah tidak menganulir gol pertama AC Milan yang dicetak Davide Calabria. Padahal, sebelum gol terjadi, Hakan Calhanoglu terlihat jelas melanggar Adrien Rabiot. Kedua, AC Milan seharusnya mendapat hadiah penalti pada menit-menit akhir usai Brahim Diaz dijatuhkan Rabiot. Namun, Irrati bergeming. Pengadil lapangan berusia 41 tahun itu tetap melanjutkan pertandingan. Baca juga: Chi

ValueError: Unable to parse line 1: KOMPAS.com - Kepemimpinan wasit Massimilano Irrati pada laga  AC Milan vs  Juventus semalam tengah menjadi sorotan. Beberapa media Italia bahkan memberi penilaian jelek terhadap kinerja wasit asal Florence dalam partai tersebut. La Gazzetta dello Sport dan Corriere dello Sport memberi nilai 4,5 untuk Irrati, sedangkan Tuttosport hanya 4. Irrati dinilai banyak membuat keputusan yang merugikan dalam grande partita pekan ke-16  Liga Italia 2020-2021 itu. Baca juga: Pahlawan Kemenangan Juventus atas AC Milan Mengaku Dihina oleh Wasit Kesalahan pertama Irrati adalah tidak menganulir gol pertama AC Milan yang dicetak Davide Calabria. Padahal, sebelum gol terjadi, Hakan Calhanoglu terlihat jelas melanggar Adrien Rabiot. Kedua, AC Milan seharusnya mendapat hadiah penalti pada menit-menit akhir usai Brahim Diaz dijatuhkan Rabiot. Namun, Irrati bergeming. Pengadil lapangan berusia 41 tahun itu tetap melanjutkan pertandingan. Baca juga: Chiesa Soal Gol Cantik L